<a href="https://colab.research.google.com/github/amorelo01/IA_UdeA_AndresZ/blob/main/arboldeeleccionGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore') 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
train = pd.read_csv('https://drive.google.com/uc?id=1OIsB4KWW429YJuwTMirBh8Y0jUbWwHwl')
test = pd.read_csv('https://drive.google.com/uc?id=1m0CDh7iom9OohCpumZLsuM4qGixartab')
submission = pd.read_csv('https://drive.google.com/uc?id=19kVehCVH3L77Rw93kHznZ2btyhC5B-zO')

In [5]:
train['target'].value_counts()

0    220539
1     79461
Name: target, dtype: int64

In [6]:
train['target'].value_counts(normalize=True)

0    0.73513
1    0.26487
Name: target, dtype: float64

In [8]:
data_all = pd.concat([train, test], sort=False)
data_all.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0.0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0.0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0.0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0.0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1.0


In [9]:
data_all = data_all.drop(columns='id')

In [10]:
data_all.shape

(500000, 31)

In [11]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 199999
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    500000 non-null  object 
 1   cat1    500000 non-null  object 
 2   cat2    500000 non-null  object 
 3   cat3    500000 non-null  object 
 4   cat4    500000 non-null  object 
 5   cat5    500000 non-null  object 
 6   cat6    500000 non-null  object 
 7   cat7    500000 non-null  object 
 8   cat8    500000 non-null  object 
 9   cat9    500000 non-null  object 
 10  cat10   500000 non-null  object 
 11  cat11   500000 non-null  object 
 12  cat12   500000 non-null  object 
 13  cat13   500000 non-null  object 
 14  cat14   500000 non-null  object 
 15  cat15   500000 non-null  object 
 16  cat16   500000 non-null  object 
 17  cat17   500000 non-null  object 
 18  cat18   500000 non-null  object 
 19  cont0   500000 non-null  float64
 20  cont1   500000 non-null  float64
 21  cont2   50

In [12]:
data_all.isnull().sum()

cat0           0
cat1           0
cat2           0
cat3           0
cat4           0
cat5           0
cat6           0
cat7           0
cat8           0
cat9           0
cat10          0
cat11          0
cat12          0
cat13          0
cat14          0
cat15          0
cat16          0
cat17          0
cat18          0
cont0          0
cont1          0
cont2          0
cont3          0
cont4          0
cont5          0
cont6          0
cont7          0
cont8          0
cont9          0
cont10         0
target    200000
dtype: int64

In [13]:
numerical_columns = data_all.drop(columns='target').select_dtypes(exclude=['object']).columns
numerical_columns

Index(['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10'],
      dtype='object')

In [14]:
categorical_columns = data_all.select_dtypes(include=['object']).columns
categorical_columns

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16',
       'cat17', 'cat18'],
      dtype='object')

In [16]:
le = LabelEncoder()
for col in categorical_columns.tolist():
  data_all[col] = le.fit_transform(data_all[col])

In [17]:
train_data = data_all.iloc[:len(train)]
test_data = data_all.iloc[len(train):].drop(columns='target')

X = train_data.drop(columns='target')
y = train_data['target']

In [18]:
from sklearn.model_selection import train_test_split

X_data_train, X_data_valid, y_data_train, y_data_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [19]:
import lightgbm as lgb

model = lgb.LGBMClassifier()
model.fit(X_data_train, y_data_train)

y_pred = model.predict(X_data_valid)

y_pred_prob = model.predict_proba(X_data_valid)

In [20]:
df_pred = pd.DataFrame({'target': y_data_valid, 'target_pred': y_pred})
df_pred.head()

,target,target_pred
157229,0.0,0.0
229228,0.0,0.0
154953,1.0,1.0
271074,0.0,0.0
270323,0.0,0.0


In [21]:
df_pred_prob = pd.DataFrame({'target': y_data_valid, 'class_0_prob': y_pred_prob[:, 0], 'class_1_prob': y_pred_prob[:, 1]})
df_pred_prob.head()

,target,class_0_prob,class_1_prob
157229,0.0,0.759767,0.240233
229228,0.0,0.922650,0.077350
154953,1.0,0.122721,0.877279
271074,0.0,0.985895,0.014105
270323,0.0,0.892291,0.107709


In [24]:
acc = accuracy_score(y_data_valid, y_pred)
print('accuracy: ', acc)

logloss = log_loss(y_data_valid, y_pred_prob)
print('logloss: ', logloss)

auc = roc_auc_score(y_data_valid, y_pred_prob[:, 1])
print('AUC: ', auc)

accuracy:  0.84875
logloss:  0.3531858381876741
AUC:  0.888497899271155


In [26]:
submission = model.predict_proba(test_data)
submission = submission[:, 1]
submission[:10]

array([0.11223662, 0.54708954, 0.04471024, 0.16731676, 0.14467897,
       0.18198867, 0.10809571, 0.0967108 , 0.26828327, 0.04221856])

In [27]:
id_column = test['id']

In [28]:
submission_df = pd.DataFrame(id_column)
submission_df['target'] = submission
submission_df.head()

,id,target
0,5,0.112237
1,6,0.547090
2,8,0.044710
3,9,0.167317
4,11,0.144679


In [29]:
submission_df.to_csv('submission.csv', index=False)